# Power Flow Study
*Copyright (c) 2023 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*
## Background

Power flow is an important numerical analysis in a power system, and it aims to calculate the various parameters of each node in a power system according to the given conditions and constraints. Power flow study is the foundation for almost all the power system analysis and evaluatons, and it also underpins the further planning, expansion, and operation determining for a power system. A simple example is that we can check whether the proposed power system planning scheme can meet the requirements through power flow calculation. Therefore, power flow calculation has a very important practical significance.

In a power flow problem, each part of a power system is represented by nodes and lines connecting nodes. For AC power, there are generally four parameters to be considered for each node, namely voltage magnitute, phase angle, active power and reactive power. Active power refers to the power of the system that actually converts the electric energy into other forms of energy, while reactive power is the power continuously flows back and forth between the source and load. According to the given initial conditions, we can obtain the unknown parameters on each node by solving the power balance equations. From the perspective of mathematical model, power flow problem can be reduced to solving a series of nonlinear equations. Speficially, for the $i$ th node in a power system with $n$ ndoes, we have:

$$\begin{cases}
P_i-U_i\sum^n_{j=1}U_j(G_{ij}\cos\delta_{ij}+B_{ij}\sin\delta_{ij}) &= 0 \\
Q_i-U_i\sum^n_{j=1}U_j(G_{ij}\sin\delta_{ij}-B_{ij}\cos\delta_{ij}) &= 0 
\end{cases}$$
where $P_i$ is the injective active power of $i$ th node, $Q_i$ is the injective reactive power of $i$ th node, and $U_i$ is the voltage magnitude of $i$ th node. Meanwhile, $G$ represents the real part of admittance matrix, $B$ represents the imaginary part of the admittance matrix, $\delta_ {ij}$ represents the phase difference between the $i$ th and $j$ th nodes.

Solving nonlinear equations is a difficult task, so we usually use some numerical methods to obtain approximate solutions. One of the most commonly used methods is Newton-Raphson method, which can gradually approach the precise solution of the equations by increasing the number of iterations. In short, Newton-Raphson method is to find the appropriate optimization direction from an initial point by calculating the derivative, and then continuously update our approximate solution to get a result that is close to the precise solution.

## Quantum solutions
We have known that Newton-Raphson method is commonly used in power flow problem. Its complexity mainly comes from the process of calculating the inversion of Jacobian matrix, and the complexity is $\mathcal{O}(poly(n))$. In Newton-Raphson method, calculating the inversion of Jacobian matrix can also be seen as the process of solving linear equations. In the field of quantum computing, many quantum algorithms have been proposed to solve linear equations e.g. Harrow-Hassidim-Lloyd (HHL) algorithm and Variational Quantum Linear Solver (VQLS). Compared with classical algorithms, they have been proved to have the exponential acceleration in solving linear equations under certain conditions. Here, we would use the variational quantum linear solver (VQLS) to replace the Jacobian matrix inversion process in Newton-Raphson method, so as to achieve the quantum acceleration of power flow problems [1]. We also provide a model and a tutorial of VQLS in our Quantum Application Model Library, you can find more details of this algorithm in its tutorial. Next, we will take a 5-node power system as an example to demonstrate the quantum solution for power flow problem based on Newton-Raphson method and VQLS.

## User's guide

We provide a configuration file with previously chosen parameters. The user just needs to change the parameters in the `config.toml` file, and run `python power_flow.py --config config.toml` in the terminal, to solve the power flow problem.
### Output
The results of power flow problem will be output to the `pf_result.txt` file. And the process of optimization will be documented in the `power_flow.log` file. Users can check the evolution of loss and error value as the number of looping iterations increases.

### Online demonstration
Here, we demonstrate a demo that can be tested online. We firstly define the contents of the configuration file:

In [9]:
test_toml = r"""
# The path of the input data. It should be a .txt file and in the IEEE Common Data Format.
data_dir = './ieee5cdf.txt'

# Threshold for loss value to end optmization for power flow, default is 1e-3
threshold = 1e-3

# Minimum number of iterations of power flow optimization.
minIter = 3

# Maximum number of iterations of power flow optimization.
maxIter = 100

# The depth of the quantum ansatz circuit.
depth = 4

# Number optimization cycles of quantum circuit. 
iterations = 100

# The learning rate of the optimizer.
LR = 0.1

# Threshold for loss value to end optimization for quantum circuit early, default is 0.
gamma = 0

"""

Then we present the code for the model：

In [8]:
import os
import warnings

warnings.filterwarnings('ignore')
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

import toml
from paddle_quantum.data_analysis.power_flow import data_to_Grid

config = toml.loads(test_toml)
file_name = config.pop('data_dir')
grid = data_to_Grid(file_name)
grid.powerflow(**config)
print("Here is the power flow results:")
grid.printResults()
Error = grid.tolerances[-1] 
print(f"Error: {Error}")
grid.saveResults()

100%|██████████| 100/100 [01:00<00:00,  1.65it/s]

Here is the power flow results:
Power Flow:

| Bus |    Bus     |    V     |  Angle   |      Injection      |      Generation     |        Load        |
| No  |    Name    |    pu    |  Degree  |     MW   |   MVar   |     MW   |  Mvar    |     MW  |     MVar |
|   1 |Bus 1     HV|    1.060 |    0.000 |  129.816 |   24.447 |  129.816 |   24.447 |   0.000 |    0.000 |
|   2 |Bus 2     HV|    1.036 |   -0.046 |   20.000 |   20.000 |   40.000 |   30.000 |  20.000 |   10.000 |
|   3 |Bus 3     HV|    1.009 |   -0.084 |  -45.000 |  -15.000 |    0.000 |    0.000 |  45.000 |   15.000 |
|   4 |Bus 4     HV|    1.007 |   -0.090 |  -40.000 |   -5.000 |   -0.000 |   -0.000 |  40.000 |    5.000 |
|   5 |Bus 5     HV|    1.002 |   -0.104 |  -60.000 |  -10.000 |    0.000 |    0.000 |  60.000 |   10.000 |
----------------------------------------------------------------------------------------------------------

Network and losses:

|  From |    To |     P     |     Q     |  From |    To |     P     | 

The error here represents the maximum absolute error of nonlinear equations.

### Note

The input data should be in the IEEE common format [2].

## Citation

```
@article{liu2022quantum,
  title={Quantum Power Flows: From Theory to Practice},
  author={Liu, Junyu and Zheng, Han and Hanada, Masanori and Setia, Kanav and Wu, Dan},
  journal={arXiv preprint arXiv:2211.05728},
  year={2022}
}

@article{pierce1973common,
  title={Common format for exchange of solved load flow data},
  author={Pierce, HE and others},
  journal={IEEE Transactions on Power Apparatus and Systems},
  volume={92},
  number={6},
  pages={1916--1925},
  year={1973}
}
```

## References
[1] Liu, Junyu, et al. "Quantum Power Flows: From Theory to Practice." arXiv preprint arXiv:2211.05728 (2022).

[2] Pierce, H. E. "Common format for exchange of solved load flow data." IEEE Transactions on Power Apparatus and Systems 92.6 (1973): 1916-1925.